# TRANSFORMACIÓN DE DATOS

## 1. IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

#Mostrar el máximo de columnas posibles de una tabla
pd.set_option('display.max_columns', None) #Número de columnas que deben verse. None = Máx


#Mostrar el máximo de filas posibles de una tabla
pd.set_option('display.max_rows', 100) #Número de filas que deben verse. None = Máx

## 2. IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
#Abrir el excel de las transformaciones que le vamos a hacer a los datos para hacer un esquema de aplicacion y no perdernos
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = True)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num).reset_index(drop = True)

In [5]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    18139 non-null  object
 1   host_response_time      18139 non-null  object
 2   host_is_superhost       18139 non-null  object
 3   host_verifications      18139 non-null  object
 4   host_has_profile_pic    18139 non-null  object
 5   host_identity_verified  18139 non-null  object
 6   property_type           18139 non-null  object
 7   room_type               18139 non-null  object
 8   amenities               18139 non-null  object
 9   has_availability        18139 non-null  object
 10  license                 18139 non-null  object
 11  instant_bookable        18139 non-null  object
 12  neighbourhood_group     18139 non-null  object
 13  host_since              18139 non-null  object
 14  first_review            18139 non-null  object
 15  la

In [6]:
#Eliminamos las variables discretizadas previamente para hacer las pruebas de predicción
cat = cat.drop(columns=['bedrooms_disc','beds_disc','number_of_reviews_disc','accommodates_disc'])

In [7]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    18139 non-null  object
 1   host_response_time      18139 non-null  object
 2   host_is_superhost       18139 non-null  object
 3   host_verifications      18139 non-null  object
 4   host_has_profile_pic    18139 non-null  object
 5   host_identity_verified  18139 non-null  object
 6   property_type           18139 non-null  object
 7   room_type               18139 non-null  object
 8   amenities               18139 non-null  object
 9   has_availability        18139 non-null  object
 10  license                 18139 non-null  object
 11  instant_bookable        18139 non-null  object
 12  neighbourhood_group     18139 non-null  object
 13  host_since              18139 non-null  object
 14  first_review            18139 non-null  object
 15  la

4.- Separar la target. En este caso hemos separado la target previamente, por lo que la cargamos desde la base de datos y trabajamos con ella

In [8]:
target = num[['pisos_rentables']].copy().reset_index(drop=True)
target

,pisos_rentables
0,1
1,1
2,1
3,0
4,1
...,...
18134,0
18135,0
18136,0
18137,0


In [12]:
print("Valores únicos de target:", num.pisos_rentables.unique())
print("Conteo de clases:", num.pisos_rentables.value_counts())

Valores únicos de target: [1 0]
Conteo de clases: pisos_rentables
1    12646
0     5493
Name: count, dtype: int64


## 3. TRANSFORMACIÓN DE CATEGÓRICAS

### 3.1. One Hot Encoding

#### 3.1.1. Variables a aplicar OHE

In [9]:
var_ohe = ['property_type',
           'room_type',
           'neighbourhood_group']

#### 3.1.2. Instanciar

In [10]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### 3.1.3. Entrenar y aplicar

In [11]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### 3.1.4. Guardar como dataframe

In [12]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### 3.2. Ordinal Encoding

#### 3.2.1. Variables a aplicar OE

In [13]:
var_oe = ['host_response_time',
          'host_is_superhost',
          'host_verifications',
          'host_has_profile_pic',
          'host_identity_verified',
          'has_availability',
          'license',
          'instant_bookable']

#### 3.2.2. Orden de los valores de las variables

In [14]:
#Orden de la primera variable
orden_host_response_time = ['no response', 'a few days or more','within a day', 'within a few hours', 'within an hour' ]

#Orden de la segunda variable
orden_host_is_superhost = ['f','t']

#Orden de la variable
orden_host_verifications = ["['No']", "['email']", "['phone']", "['phone', 'work_email']", "['email', 'phone']", "['email', 'phone', 'work_email']"]

#Orden de la variable
orden_host_has_profile_pic = ['f','t']

#Orden de la variable
orden_host_identity_verified = ['f','t']

#Orden de la variable
orden_has_availability = ['yes','no']

#Orden de la variable
orden_license = ['no','yes']

#Orden de la variable
orden_instant_bookable = ['f','t']

#### 3.2.3. Instanciar

In [15]:
oe = OrdinalEncoder(categories = [orden_host_response_time,
                                  orden_host_is_superhost,
                                  orden_host_verifications,
                                  orden_host_has_profile_pic,
                                  orden_host_identity_verified,
                                  orden_has_availability,
                                  orden_license,
                                  orden_instant_bookable                                 ],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 10)

#### 3.2.4. Entrenar y aplicar

In [16]:
cat_oe = oe.fit_transform(cat[var_oe])

#### 3.2.5. Guardar como dataframe

In [17]:
#Añadir sufijos a los nombres
nombres_oe = [variable + '_oe' for variable in var_oe]

#Guardar como dataframe
cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)
cat_oe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_response_time_oe      18139 non-null  float64
 1   host_is_superhost_oe       18139 non-null  float64
 2   host_verifications_oe      18139 non-null  float64
 3   host_has_profile_pic_oe    18139 non-null  float64
 4   host_identity_verified_oe  18139 non-null  float64
 5   has_availability_oe        18139 non-null  float64
 6   license_oe                 18139 non-null  float64
 7   instant_bookable_oe        18139 non-null  float64
dtypes: float64(8)
memory usage: 1.1 MB


In [18]:
print(cat_oe.dtypes)
print(cat_oe.head())

host_response_time_oe        float64
host_is_superhost_oe         float64
host_verifications_oe        float64
host_has_profile_pic_oe      float64
host_identity_verified_oe    float64
has_availability_oe          float64
license_oe                   float64
instant_bookable_oe          float64
dtype: object
   host_response_time_oe  host_is_superhost_oe  host_verifications_oe  \
0                    0.0                   0.0                    4.0   
1                    0.0                   0.0                    4.0   
2                    0.0                   0.0                    4.0   
3                    4.0                   0.0                    4.0   
4                    0.0                   0.0                    4.0   

   host_has_profile_pic_oe  host_identity_verified_oe  has_availability_oe  \
0                      1.0                        1.0                  0.0   
1                      1.0                        0.0                  1.0   
2                

### 3.4. Fechas y textos

In [19]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18139 entries, 0 to 18138
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    18139 non-null  object
 1   host_response_time      18139 non-null  object
 2   host_is_superhost       18139 non-null  object
 3   host_verifications      18139 non-null  object
 4   host_has_profile_pic    18139 non-null  object
 5   host_identity_verified  18139 non-null  object
 6   property_type           18139 non-null  object
 7   room_type               18139 non-null  object
 8   amenities               18139 non-null  object
 9   has_availability        18139 non-null  object
 10  license                 18139 non-null  object
 11  instant_bookable        18139 non-null  object
 12  neighbourhood_group     18139 non-null  object
 13  host_since              18139 non-null  object
 14  first_review            18139 non-null  object
 15  la

In [20]:
def modificaciones_fechas(cat):
    
    def convertir_fecha(fecha):
        for formato in ['%Y-%m-%d', '%d/%m/%Y', '%d-%m-%Y', '%Y-%d-%m', '%m/%d/%Y']:
            try:
                return pd.to_datetime(fecha, format=formato)
            except ValueError:
                continue
        return pd.NaT  # Si no coincide con ningún formato

    # Aplicar la función a la columna, asegurando que el DataFrame se mantenga
    cat['host_since'] = cat['host_since'].apply(convertir_fecha)
    cat['first_review'] = cat['first_review'].apply(convertir_fecha)
    cat['last_review'] = cat['last_review'].apply(convertir_fecha)

    # Convertir las fechas al formato deseado y sobrescribir la columna original
    cat['host_since'] = pd.to_datetime(cat['host_since'], dayfirst=True, errors='coerce')
    cat['first_review'] = pd.to_datetime(cat['first_review'], dayfirst=True, errors='coerce')
    cat['last_review'] = pd.to_datetime(cat['last_review'], dayfirst=True, errors='coerce')

    def componentes_fecha(cat):

        cat.reset_index(inplace=True)

        #host_since
        cat.set_index('host_since', inplace=True)    
        cat['host_since_year'] = cat.index.year

        #first_review
        cat.set_index('first_review', inplace=True)    
        cat['first_review_year'] = cat.index.year

        #last_review
        cat.set_index('last_review', inplace=True)    
        cat['last_review_year'] = cat.index.year

        #cat.reset_index(inplace=True)

        return(cat)
    cat = componentes_fecha(cat)
    cat['host_since_year'] = cat['host_since_year'].astype('O')
    cat['first_review_year'] = cat['first_review_year'].astype('O')
    cat['last_review_year'] = cat['last_review_year'].astype('O')
    return(cat)

cat = modificaciones_fechas(cat)

In [25]:
def variables_texto(cat):    
    #Cargamos la lista de stopwords en un dataframe. La lista la tendremos que descargar y ubicar antes de hacer nada 
    stopwords = pd.read_csv('C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/01_Documentos/stop_words_spanish.txt',names = ['termino'])

    #Función para quitar todas las tides de las vocales
    def quitar_tildes(palabra):
        #Definimos la versión con tildes y símbolos y la sin
        con = 'áéíóúüñÁÉÍÓÚÜÑ'
        sin = 'aeiouunAEIOUUN'
        #Creamos un traductor
        traductor = str.maketrans(con,sin)
        #Aplicamos el traductor y devolvemos la palabra limpia
        return(palabra.translate(traductor))

    #Creamos una nueva columna en el dataframe de stopwords llamada 'limpias'
    stopwords['limpias'] = stopwords.termino.transform(quitar_tildes)

    #Cargamos el modelo
    from sklearn.feature_extraction.text import CountVectorizer

    #Instanciamos
    cv = CountVectorizer(strip_accents = 'unicode',
                         stop_words = stopwords.limpias.to_list(),
                         max_df = 0.7,
                         min_df = 50,
                         max_features = 50,
                         ngram_range = (1,3),#Contemplará como términos correctos los que estén compuestos por 1, 2 y 3 palabras
                         binary=True)

    #Entrenamos
    cv.fit(cat.amenities)
    
    # Guarda a disco como modelo para que la separación de los términos sea siempre la misma
    with open("count_vectorizer.pkl", "wb") as f:
        pickle.dump(cv, f)       

    # Cargar el vectorizador guardado
    with open("count_vectorizer.pkl", "rb") as f:
        cv = pickle.load(f)    

    #Aplicamos
    caracteristicas = cv.transform(cat.amenities)
    caracteristicas

    #Podemos ver el vocabulario total encontrado y su número de columna.
    tt = cv.vocabulary_
    tt

    #Para pasar a dataframe el vocabulario encontrado con sus nombres de columnas y ordenado
    vo = pd.DataFrame(list(tt.items()), columns=['Termino', 'Indice']).sort_values('Indice')
    vo

    #Definimos un dataframe con los términos encontrados transformados a variables que mapean las columnas donde se encuentran
    terminos = pd.DataFrame(caracteristicas.toarray(), columns=cv.get_feature_names_out())
    return(terminos)
df_terminos = variables_texto(cat)
df_terminos



,air,air conditioning,allowed,and silverware,basics,bed,bed linens,body,body soap,cleaning,clothing,clothing storage,coffee,coffee maker,conditioning,cooking,cooking basics,dedicated,dedicated workspace,dishes,dishes and,dishes and silverware,dryer bed,dryer bed linens,elevator,freezer,hair,hair dryer,hair dryer bed,hot water iron,in,iron,kitchen essentials,linens,maker,microwave,microwave hangers,oven,parking,refrigerator,room,shampoo,silverware,soap,storage,stove,tv washer,water iron,wifi kitchen,workspace
0,1,1,0,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,1,0,1
4,0,0,0,1,1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18134,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1
18135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18136,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1
18137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [26]:
list(df_terminos)

['air',
 'air conditioning',
 'allowed',
 'and silverware',
 'basics',
 'bed',
 'bed linens',
 'body',
 'body soap',
 'cleaning',
 'clothing',
 'clothing storage',
 'coffee',
 'coffee maker',
 'conditioning',
 'cooking',
 'cooking basics',
 'dedicated',
 'dedicated workspace',
 'dishes',
 'dishes and',
 'dishes and silverware',
 'dryer bed',
 'dryer bed linens',
 'elevator',
 'freezer',
 'hair',
 'hair dryer',
 'hair dryer bed',
 'hot water iron',
 'in',
 'iron',
 'kitchen essentials',
 'linens',
 'maker',
 'microwave',
 'microwave hangers',
 'oven',
 'parking',
 'refrigerator',
 'room',
 'shampoo',
 'silverware',
 'soap',
 'storage',
 'stove',
 'tv washer',
 'water iron',
 'wifi kitchen',
 'workspace']

### 4.1. Discretizar variables

#### 4.1.1. Mediante distancia similar

##### Variables a discretizar mediante distancia similar

In [27]:
num.info()

In [28]:
var_disc_ds = ['bathrooms',
 'price',
 'minimum_nights',
 'maximum_nights',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'accommodates',
 'beds',
 'bedrooms',
 'number_of_reviews']

##### Instanciar

In [29]:
disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

##### Entrenar y aplicar

In [30]:
num_disc_ds = disc_ds.fit_transform(num[var_disc_ds])

##### Guardar como dataframe

In [31]:
#Añadir sufijos a los nombres
nombres_ds = [variable + '_disc_ds' for variable in var_disc_ds]

#Guardar como dataframe
num_disc_ds = pd.DataFrame(num_disc_ds,columns = nombres_ds)

### Normalizar (Gauss)

#### Con Yeo-Johnson

##### Variables a normalizar con Yeo-Johnson

In [32]:
var_yeo = ['host_response_rate',
           'accommodates',
           'maximum_nights',
           'price',
           'availability_30',
           'availability_60',
           'availability_90',
           'availability_365']

##### Instanciar

In [33]:
yeo = PowerTransformer(method = 'yeo-johnson')

##### Entrenar y aplicar

In [34]:
num_yeo = yeo.fit_transform(num[var_yeo])

##### Guardar como dataframe

In [35]:
#Añadir sufijos a los nombres
nombres_yeo = [variable + '_yeo' for variable in var_yeo]

#Guardar como dataframe
num_yeo = pd.DataFrame(num_yeo,columns = nombres_yeo)

## 5. UNIFICAR DATASETS TRANSFORMADOS

### 5.1. Meter en una lista todos los dataframes generados

In [36]:
dataframes = []
dataframes.extend(value for name, value in locals().items() if name.startswith('cat_') or name.startswith('num_'))

### 5.2. Unir todos los dataframes

In [37]:
df = pd.concat(dataframes, axis = 1)

In [38]:
df

## 6.  REESCALAR VARIABLES

### 6.1. Con Min-Max

In [39]:
#Sacamos un info porque lo ideal es reescalar las variables que no hayan sido resultado de un OneHoteEncoding
list(df)

#### 6.1.1. Variables a reescalar con Min-Max

In [40]:
var_mms = ['host_response_time_oe',
 'host_is_superhost_oe',
 'host_verifications_oe',
 'host_has_profile_pic_oe',
 'host_identity_verified_oe',
 'has_availability_oe',
 'license_oe',
 'instant_bookable_oe',
 'bathrooms_disc_ds',
 'price_disc_ds',
 'minimum_nights_disc_ds',
 'maximum_nights_disc_ds',
 'calculated_host_listings_count_disc_ds',
 'calculated_host_listings_count_entire_homes_disc_ds',
 'calculated_host_listings_count_private_rooms_disc_ds',
 'calculated_host_listings_count_shared_rooms_disc_ds',
 'accommodates_disc_ds',
 'beds_disc_ds',
 'bedrooms_disc_ds',
 'number_of_reviews_disc_ds',
 'host_response_rate_yeo',
 'accommodates_yeo',
 'maximum_nights_yeo',
 'price_yeo',
 'availability_30_yeo',
 'availability_60_yeo',
 'availability_90_yeo',
 'availability_365_yeo']

#### 6.1.2. Instanciar

In [41]:
mms = MinMaxScaler()

#### 6.1.3. Entrenar y aplicar

In [42]:
df_mms = mms.fit_transform(df[var_mms])

#### 6.1.4. Guardar como dataframe

In [43]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]
#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)
df_mms

## 7. UNIFICAR DATASETS REESCALADOS

### 7.1. Crear una lista con los dataframes a incluir en el tablón analítico

In [44]:
incluir = [df, 
           df_mms, #df MinMax Scaler
           #df_rs, #df Robust Scaler
           #df_ss, #df Standard Scaler
           df_terminos,
           target
           ]

### 7.2. Unir todos los dataframes en el tablón analítico

In [45]:
df_tablon = pd.concat(incluir, axis = 1)

In [46]:
#Detectamos que se han generado 7774 nulos 
df_tablon.isna().sum().head()

In [47]:
criterio = 3 #Mínimo de variables SIN nulos por registro. Se elimina el registro si hay menos de esas variables sin nulos
df_tablon.dropna(thresh=criterio, inplace=True)
df_tablon.isna().sum().sort_values(ascending=True)

## 8. GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [48]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [49]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)

In [50]:
df_tablon